In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torch_geometric as pyg
import torch_geometric.datasets as pygd
import gpn.experiments.transductive_experiment as exp

In [69]:
from gpn.utils.config import DataConfiguration, ModelConfiguration, RunConfiguration, TrainingConfiguration


run_cfg = RunConfiguration(
    job="train",
    eval_mode="default",
    experiment_directory="./saved_experiments",
    save_model=True,
    gpu=0,
    experiment_name="ood_loc"
)
data_cfg = DataConfiguration(
    dataset="CoraML",
    split_no=1,
    root="./data",
    ood_flag=True,
    train_samples_per_class=0.05,
    val_samples_per_class=0.15,
    test_samples_per_class=0.8,
    split="random",
    ood_setting="poisoning",
    ood_type="leave_out_classes",
    ood_num_left_out_classes=-1,
    ood_leave_out_last_classes=True
)

model_cfg = ModelConfiguration(
    model_name="GPN_LOP",
    seed=42,
    init_no=1,
    dim_hidden=64,
    dropout_prob=0.5,
    K=10,
    add_self_loops=True,
    maf_layers=0,
    gaussian_layers=0,
    use_batched_flow=True,
    loss_reduction="sum",
    approximate_reg=True,
    flow_weight_decay=0.0,
    pre_train_mode="flow",
    alpha_evidence_scale="latent-new-plus-classes",
    alpha_teleport=0.2,
    entropy_reg=1.0e-05,
    dim_latent=10,
    radial_layers=10
)

train_cfg = TrainingConfiguration(
    epochs=100000,
    stopping_mode="default",
    stopping_patience=50,
    stopping_restore_best=True,
    stopping_metric="val_CE",
    stopping_minimize=True,
    finetune_epochs=0,
    warmup_epochs=5,
    lr=0.01,
    weight_decay=0.0005
)

e = exp.TransductiveExperiment(run_cfg, data_cfg, model_cfg, train_cfg)
e

In [70]:
run_cfg.job

'evaluate'

In [60]:
res = e.run()

/home/clemens/Projects/Graph-Posterior-Network/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1124: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/home/clemens/Projects/Graph-Posterior-Network/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1124: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(


In [ ]:
from gpn.utils.utils import results_dict_to_df

results_dict_to_df(res)

In [ ]:
a = torch.tensor([[2, 8], [20, 80], [200, 800]], dtype=torch.float32)

w = torch.tensor([[0.5, 0.5, 0], [0, 1, 0], [0, 0.5, 0.5]])

y = torch.tensor([1, 0, 1])

mix_sum = w @ a.sum(-1)

(w @ a), (w @ a).gather(-1, y.view(-1, 1)).squeeze(-1)